In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
from langchain_teddynote import logging

logging.langsmith("[Training] rag_basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
[Training] rag_basic


In [4]:
from dart_document_loader import DartDocumentLoader

loader = DartDocumentLoader("20240320001808")
docs = loader.load()

In [5]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
# split_documents = docs

In [6]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [7]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [8]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [9]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [10]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [11]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "제너셈의 주요 제품 및 제품 성장과정에 대해 알아야 하는 주요 내용들에 대한 엔티티를 추출해주세요."
response = chain.invoke(question)
print(response)

제너셈의 주요 제품 및 제품 성장과정에 대해 알아야 하는 주요 내용들에 대한 엔티티는 다음과 같습니다:

1. **주요 제품**:
   - 반도체 장비

2. **제품 성장과정**:
   - **설계 도면 작성**: 시스템개발기술그룹에서 설계 도면 작성
   - **부품 조달**: 각 부품업체와 설계 도면 공유 후 부품 발주
   - **장비 제작**: 고객사와 사양, 납기, 가격 협의 후 장비 제작 시작 (2~3개월 소요)
   - **외주생산**: 외주업체를 이용하여 장비 제작

3. **연구개발 활동**:
   - **연구개발 조직**: 제너셈 R&D센터
   - **주요 연구 분야**: Laser, vision을 이용한 응용기술 개발 및 국책과제 수립
   - **연구개발비용**: 제24기 4,385백만원, 제23기 4,161백만원

4. **매출 정보**:
   - **지역별 매출**: 국내, 멕시코, 아시아, 기타
   - **주요 고객**: 당기 매출의 10% 이상 차지하는 고객

이와 같은 엔티티들이 제너셈의 주요 제품 및 제품 성장과정에 대한 주요 내용들입니다.
